In [1]:
import pandas as pd
import os
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam


In [2]:
df=pd.read_csv('training_set_pixel_size_and_HC.csv')

images = []
hcs = []

for _, row in df.iterrows():
    image_filename = row['filename']
    image_path = os.path.join('training_set', image_filename)

    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (800, 540))
    images.append(img_resized)
    hcs.append(row['head circumference (mm)'])
 
images = np.array(images, dtype='float32')
images = np.expand_dims(images, -1)
hcs = np.array(hcs, dtype='float32')

images /= 255.0

train_images, val_images, train_hcs, val_hcs = train_test_split(images, hcs, test_size=0.2, random_state=69)

In [3]:
def regression_network(input_shape):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])
    return model

In [4]:
input_shape = (540, 800, 3)
regression_model = regression_network(input_shape)

In [ ]:
checkpoint_callback = ModelCheckpoint(filepath='best_model.h5', 
                                      monitor='val_loss', 
                                      save_best_only=True)

history = regression_model.fit(train_images, train_hcs,
                    validation_data=(val_images, val_hcs),
                    epochs=15, batch_size=32, callbacks=[checkpoint_callback])

Epoch 1/15


25/25 [==============================] - ETA: 0s - loss: 10203.5986 - mae: 78.6870

C:\Users\nguye\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/25 [==============================] - 23s 877ms/step - loss: 10203.5986 - mae: 78.6870 - val_loss: 2976.5632 - val_mae: 41.8341
Epoch 2/15
25/25 [==============================] - 21s 848ms/step - loss: 3537.7053 - mae: 46.6445 - val_loss: 2075.1462 - val_mae: 35.1594
Epoch 3/15
25/25 [==============================] - 20s 808ms/step - loss: 2816.6299 - mae: 40.6536 - val_loss: 2084.4941 - val_mae: 34.6205
Epoch 4/15
25/25 [==============================] - 21s 837ms/step - loss: 2711.0898 - mae: 41.3183 - val_loss: 1521.0361 - val_mae: 30.5481
Epoch 5/15
25/25 [==============================] - 20s 807ms/step - loss: 2125.5918 - mae: 35.9489 - val_loss: 1643.6953 - val_mae: 30.8919
Epoch 6/15
25/25 [==============================] - 21s 843ms/step - loss: 1857.9017 - mae: 33.5838 - val_loss: 1282.7454 - val_mae: 27.8657
Epoch 7/15
25/25 [==============================] - 21s 846ms/step - loss: 1785.5570 - mae: 32.5211 - val_loss: 1263.5099 - val_mae: 27.3823
Epoch 8/15
25/25 [=====